# Winning Team - what are we trying to do in this kernel?

In this kernel, we will attempt to bring together a series of ideas. We attempt to: 

1. [X] Use Ben Graham's image preprocessing method
2. [X] Dynamically zoom to crop out all black edges
3. [X] Apply an additional random zoom on the zoomed-out photos of 20-40% percent. (our current models are very good at predicting class 1-4, but the training images in class 0 are different, we apply a selective zoom in order to improve our training on the 0's.
4. [X] Bring in external data as supplied by Brandon, found here: https://ieee-dataport.org/open-access/indian-diabetic-retinopathy-image-dataset-idrid 
5. [X] Use a callback to monitor and optimize for the Quadratic Weighted Kappa
6. [X] Use preprocessing to introduce flips and turns, rotation, other, to further enrich the training dataset
7. [X] Build a set of binary classifiers and compare
8. [X] Add in early stopping if the model begins to overfit
9. [X] Save the best models as they occur
10. [X] Test out various CNN architectures (EfficientNetB5, Fast-R CNN)
11. [ ] Build a multiple-input model using Nicholas' Zernike Moments
12. [ ] Linear Output on the NN and compare
13. [ ] Fix over-fitting problem
![](http://)14. [ ] Find additional external images? 

SUmmary of changes this iteration: V20:
* epochs to 15
* changed zoom to [1, 1.25]
* changed validation split from .1 to .2

V21:
* Made the Neural net Deeper

V22 or V23: 
* Added some level of rotation to the preprocessing set. 

V24: 
* To prevent overfitting and high variance on our newer, deeper, neural net, I'm adding in a small regularization factor = (l2 = 0.0005) as well as a second dropout layer (p=.25) to see if this will make a difference.
    ** Note: if this works well, it may make sense to do a GridSearch as described in this tutorial to find the optimal regularization factor.   https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/
    
    
V25: 
* Trying out EfficientnetB5!! Whoo hoo!    Never got this to work






In [ ]:
#!pip install -U '../input/efficientnetwhl/keras_efficientnet-0.1.4-py3-none-any.whl'

In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm
import cv2
from sklearn.utils import class_weight, shuffle
from random import *
import warnings
warnings.filterwarnings("ignore")
SEED = 42
IMG_SIZE = 256
# Specify title of our final model
SAVED_MODEL_NAME = 'efficient_netB7.h5'

%matplotlib inline




In [ ]:
#!pip install -U git+https://github.com/qubvel/efficientnet
#!pip install efficientnet-master
#import efficientnet

#!pip freeze

In [ ]:
import sys
import os
#sys.path.append(os.path.abspath('../input/efficientnet2ndattempt/efficientnet-master/'))
#from efficientnet.model import EfficientNet


###

sys.path.append(os.path.abspath('../input/efficientnet/efficientnet-master/efficientnet-master/'))
from efficientnet import EfficientNetB5

efficientnet_model = EfficientNetB5(weights=None, include_top=False, input_shape=(224, 224, 3))


#/kaggle/input/efficientnet2ndattempt/efficientnet-master/efficientnet/model.py
#/kaggle/input/efficientnet-keras-weights-b0b5/efficientnet-b1_imagenet_1000_notop.h5




#import EffienctNet


#import keras_efficientnet as efn
#import efficientnet.keras as efn 

#model = EfficientNetB5(weights='imagenet')
#efn.show
#import efficientnet.keras as efn 

#model = efn.EfficientNetB0(weights='imagenet')
# credit to Jenessa
#from keras.applications import MobileNet
#from keras.applications.mobilenet import preprocess_input 

#model = efn.EfficientNetB5(weights='imagenet')


#efn.show
#import efficientnet.keras as efn 
# Import the model weights pretrained on Imagenet available in the Keras framework.
# mobileNet_base = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
# Specify title of our final model
#SAVED_MODEL_NAME = 'x.h5'

In [ ]:
#!pip install -U --pre efficientnet
#EfficientNetB5(weights='imagenet')
#model = .EfficientNetB5(weights='imagenet')
#efn.show
#import efficientnet.keras as efn 

#/kaggle/input/efficientnet2ndattempt/efficientnet-master/efficientnet/model.py
#/kaggle/input/efficientnet-keras-weights-b0b5/efficientnet-b1_imagenet_1000_notop.h5

In [ ]:
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

Set random seed for reproducibility.

In [ ]:
np.random.seed(2019)
tf.set_random_seed(2019)

# Loading & Exploration

In [ ]:
#train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
###################### Adding in external dataset which has been combined with the internal data####################
train_df = pd.read_csv('/kaggle/input/ext-data/train_combined.csv')


print(train_df.shape)
print(test_df.shape)
test_df.head()

In [ ]:
train_df['diagnosis'].hist()
train_df['diagnosis'].value_counts()

### Shown below are the Class 0's, compared with other classes
Our current best model is good at predicting Classes 1, 2, 3, and 4, but is missing many of the Class 0's. This is because many of the Class 0's are zoomed too far out, and so the CNN is training on the surrounding black space instead of features of the eye itself. We hope to improve the differences on the 0's through preprocessing to make them ultimately more similar to the prediction dataset. 

In [ ]:
#Show me the 0's
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_df[train_df['diagnosis']==0].reset_index())

In [ ]:
#Show me the other classes
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_df[train_df['diagnosis']!=0].reset_index())

## Ben Graham's preprocessing method.
Ben Graham (last competition's winner) has an insightful way to improve lighting condition. Here, we apply his idea, and can see many important details in the eyes much better. 
* Credit to Neuron Engineer at https://www.kaggle.com/ratthachat/aptos-updatedv14-preprocessing-ben-s-cropping


In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        #print(check_shape)
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        #    print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
        #    print(img.shape)
        return img

In [ ]:
# Credit to Stack Overflow
def cv2_clipped_zoom(img, zoom_factor):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

# Introducing a dynamic zoom on images that are too zoomed out (mostly class 0's)

In [ ]:
#train_dir = '/kaggle/input/aptos2019-blindness-detection/train_images/'
#######################################################################################
# Pointing train_dir to include external and internal data combined
#######################################################################################
train_dir = '/kaggle/input/ext-data/train_combined/train_combined/'
imgpath = train_df[train_df['diagnosis']==0]['id_code'][4]

path = train_dir+imgpath+'.png'
image = cv2.imread(path)
plt.imshow(image)
print(image.shape[1])
orig_shape = image.shape[1]
plt.show()
plt.imshow(crop_image_from_gray(image,tol=7))
print(crop_image_from_gray(image,tol=7).shape)
cropped_shape = crop_image_from_gray(image,tol=7).shape[1]
cropping_threshold = 0.05
print((cropped_shape - orig_shape)/orig_shape)
plt.imshow(cv2_clipped_zoom(image, 1.4))


In [ ]:
def load_ben_color_for_image(image, sigmaX=10):
   # image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    orig_shape = image.shape[1]
    image = crop_image_from_gray(image,tol=7)
    cropped_shape = image.shape[1]
    # Places an additional zoom on way-zoomed out eyes
    cropping_threshold = -0.01
    zoom_level  = random()*.2+1.0
    if ((cropped_shape - orig_shape)/orig_shape < cropping_threshold):
        image = cv2_clipped_zoom(image, zoom_level) 
    
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
# Do it all preprocessing
def do_it_all(image):
    #print(image)
    # recolors, crops black space, adds the GaussianBlur, resizes
    benned_image = load_ben_color_for_image(image)
    benned_image = benned_image.astype("float32")*(1.)/255
    return np.array(benned_image)

In [ ]:
# Example of preprocessed images from every label

fig, ax = plt.subplots(1, 5, figsize=(15, 6))
for i in range(5):
    sample = train_df[train_df['diagnosis'] == i].sample(1)
    image_name = sample['id_code'].item()
    image_png = f"{image_name}.png"
    X = do_it_all(cv2.imread(f"{train_dir}{image_png}"))

    ax[i].set_title(f"Image: {image_name}\n Label = {sample['diagnosis'].item()}", 
                    weight='bold', fontsize=10)
    ax[i].axis('off')
    ax[i].imshow(X);

## Prepare the dataframe for the flow_from_dataframe function

In [ ]:
training_df = pd.DataFrame()
testing_df = pd.DataFrame()
training_df['diagnosis'] = train_df['diagnosis'].astype(str)

training_df['id'] = train_df.id_code.apply(lambda x: x + '.png')
testing_df['id'] = test_df.id_code.apply(lambda x: x + '.png')

In [ ]:
training_df.head()

In [ ]:
y_train = pd.get_dummies(training_df['diagnosis']).values
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])
testing = pd.DataFrame()
#testing['0', '1', '2', '3', '4'] = y_train_multi
print("Original y_train:\n", y_train[:3])
print("Multilabel version:\n", y_train_multi[:3])

y_train_multi_df = pd.DataFrame(data = y_train_multi[0:, 0:], index = y_train_multi[0:, 0], columns = ['0', '1', '2', '3', '4'])
y_train_multi_df = y_train_multi_df.reset_index(drop = True)
appendedTrainDF = training_df.merge(y_train_multi_df, left_index = True, right_index = True)
training_df = appendedTrainDF
print(appendedTrainDF.head())


In [ ]:

from tensorflow.python.keras.utils.data_utils import Sequence

#train_dir = '/kaggle/input/aptos2019-blindness-detection/train_images/'

##########
#  Ext data
##############

train_dir = '/kaggle/input/ext-data/train_combined/train_combined/'
datagen=ImageDataGenerator(rescale=1./255, validation_split = 0.20)

train_generator=datagen.flow_from_dataframe(
    dataframe=training_df, 
    directory=train_dir, 
    x_col="id", 
    y_col=['0', '1', '2', '3', '4'],#"diagnosis", 
    subset = 'training', 
    class_mode='raw',#"categorical", 
    target_size=(224,224), 
    batch_size=20, 
    horizontal_flip = True,
    zoom_range = [1,1.20],#0.25,
    vertical_flip = True,
    #just added
    shuffle=True,
    #Added in V29
 #   rotation_range = 120,
    preprocessing_function = do_it_all)

valid_generator=datagen.flow_from_dataframe(
    dataframe=training_df, 
    directory = train_dir, 
    x_col="id", 
    y_col=['0', '1', '2', '3', '4'],#"diagnosis", 
    subset = 'validation', 
    target_size=(224, 224), 
    batch_size=20, 
    class_mode='raw',#'categorical', 
    horizontal_flip = True,
    zoom_range = [1,1.20],#0.25,
    vertical_flip = True,
    #just added
    shuffle=True,
    #Added in V29
#    rotation_range = 120,

    preprocessing_function = do_it_all)



### Creating keras callback for Quadratic Weighted Kappa

In [ ]:
#new

def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()


class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, valid_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return

# Model: DenseNet-121

In [ ]:
#densenet = DenseNet121(
#    weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5',
#    include_top=False,
#    input_shape=(224,224,3)
#)

#import effecientnet
#efficientnetb5 = efn.EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
def build_model():
    model = Sequential()
    model.add(efficientnet_model)
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(2048))
    model.add(layers.LeakyReLU())
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    #model.add(layers.Dense(1, activation='linear'))
    
    
    
    
    
    #model.add(efficientnetb5)    
 #   model.add(layers.GlobalAveragePooling2D())
 ##   model.add(layers.Dropout(0.5))
 #   model.add(layers.Dense(1024, activation = 'relu'))#, kernel_regularizer=l2(0.0005)))
    #model.add(layers.Dense(1024, activation = 'relu'))#, kernel_regularizer=l2(0.0005)))
 #   model.add(layers.Dense(512, activation = 'relu'))#, kernel_regularizer=l2(0.001)))

    model.add(layers.Dense(5, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
#        optimizer=Adam(lr=0.00005),
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

# Training & Evaluation

In [ ]:
kappa_metrics = Metrics()

# Monitor MSE to avoid overfitting and save best model
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=1, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.0001)

BATCH_SIZE = valid_generator.batch_size
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


#from keras.utils import to_categorical

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN, 
                              validation_data=valid_generator, 
                              validation_steps=STEP_SIZE_VALID, 
                              epochs=15,
                              callbacks=[kappa_metrics, es, rlr]
)

#After 3 epochs, Epoch 3/3 With selective zoom in place
#164/164 [==============================] - 352s 2s/step - loss: 0.1890 - acc: 0.9243 - val_loss: 0.1732 - val_acc: 0.9266

#After 3 epochs Epoch 3/3 With multiple labels addition
#164/164 [==============================] - 353s 2s/step - loss: 0.1270 - acc: 0.9525 - val_loss: 0.1328 - val_acc: 0.9526

#AFter 3 epochs with random zoom set at rand()*0.2+1.3
##Epoch 3/4
#164/164 [==============================] - 388s 2s/step - loss: 0.1262 - acc: 0.9524 - val_loss: 0.1346 - val_acc: 0.9462

#Added model saving capabilities based on the QWK, added preprocessing to the test step
#164/164 [==============================] - 347s 2s/step - loss: 0.1244 - acc: 0.9535 - val_loss: 0.1288 - val_acc: 0.9506
#val_kappa: 0.8877

#Introducing a much larger range on the zoom, and from 1 to 1.4 instead of .75-.25   --> 


# Adding in the external data!!!


# Solved the overfitting problem and made neural net deeper
#Epoch 15/15
#215/215 [==============================] - 621s 3s/step - loss: 0.1470 - acc: 0.9375 - val_loss: 0.1345 - val_acc: 0.9418
#val_kappa: 0.8841x

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))


plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# Load best weights according to MSE
model.load_weights(SAVED_MODEL_NAME)

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testing_df,
    directory="/kaggle/input/aptos2019-blindness-detection/test_images/",
    x_col="id",
    y_col=None,
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(224,224), 
    preprocessing_function = do_it_all
    )

model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_VALID)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_generator.reset()
pred=model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)



## Submit

In [ ]:
y_test = pred > 0.5
y_test = y_test.astype(int).sum(axis=1) - 1

test_df['diagnosis'] = y_test
test_df.to_csv('submission.csv',index=False)

In [ ]:
# Label distribution
train_df['diagnosis'].value_counts().sort_index().plot(kind="bar", 
                                                       figsize=(15,5), 
                                                       rot=0)
plt.title("Diagnosis Distribution in the Training Set", 
          weight='bold', 
          fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Label", fontsize=17)
plt.ylabel("Frequency", fontsize=17);
plt.show()

# Label distribution
test_df['diagnosis'].value_counts().sort_index().plot(kind="bar", 
                                                       figsize=(15,5), 
                                                       rot=0)
plt.title("Label Distribution (Training Set)", 
          weight='bold', 
          fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Label", fontsize=17)
plt.ylabel("Frequency", fontsize=17);
plt.show()